# Nighborhoods in Rome Clustering
## Opening a new restaraunt in Rome

### This work is divided into 5 main categories:
- Build a dataframe of neighborhoods in Rome by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new restaurant

### Importing the libreries

In [33]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [32]:
pip install geocoder

     |████████████████████████████████| 98 kB 5.9 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
wikiurl="https://en.wikipedia.org/wiki/Category:Rome_by_rione"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [21]:
soup = BeautifulSoup(response.text, 'html.parser')
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [27]:
df=pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,"► Rome R. XIV Borgo‎ (2 C, 39 P)"
1,"► Rome R. X Campitelli‎ (2 C, 54 P)"
2,"► Rome R. IV Campo Marzio‎ (4 C, 57 P)"
3,"► Rome R. XVIII Castro Pretorio‎ (1 C, 12 P)"
4,"► Rome R. XIX Celio‎ (1 C, 15 P)"


### Deleting uesless characters

In [28]:
df["Neighborhood"] = df["Neighborhood"].map(lambda x: x.lstrip('► '))
df["Neighborhood"] = df["Neighborhood"].map(lambda x: str(x)[:-11])
df

,Neighborhood
0,Rome R. XIV Borgo‎
1,Rome R. X Campitelli‎
2,Rome R. IV Campo Marzio‎
3,Rome R. XVIII Castro Pretorio‎
4,Rome R. XIX Celio‎
5,Rome R. III Colonna‎
6,Rome R. XV Esquilino‎
7,Rome R. XVI Ludovisi‎
8,Rome R. I Monti‎
9,Rome R. VI Parione‎


Let's find the index of the row in which Bourough = Not assigned, in oreder to **drop it**

In [34]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Rome, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [35]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [37]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords

,Latitude,Longitude
0,41.903220,12.495650
1,41.893250,12.481430
2,41.907100,12.477860
3,41.905510,12.501880
4,41.903220,12.495650
5,41.836429,12.756938
6,41.894030,12.506000
7,41.907760,12.489550
8,41.893150,12.488250
9,41.897780,12.470750


In [38]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

### Saving the df in a csv because Who knows?

In [41]:
df.to_csv("rome_df.csv", index=False)

In [42]:
df.shape

(22, 3)

### 4. Create a map of Rome with neighborhoods superimposed on top

In [44]:
# get the coordinates of Rome
address = 'Rome, Italy'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rome, Italy {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rome, Italy 41.8933203, 12.4829321.


### We have found 39 Borough containing the word Toronto

In [48]:
# create map of Rome using latitude and longitude values
map_rome = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_rome)  
    
map_rome

In [49]:
# save the map as HTML file
map_rome.save('map_kl.html')

In [50]:
CLIENT_ID = 'ABUCNR5SXZYM05BZ3NGTUWUST0T4TDFSZPZP5FBX55IZJDRR' # your Foursquare ID
CLIENT_SECRET = 'ZAL0MIKYUWEA0NZDQTD4MXLPAAHZ5DRMNF4L54YW2BHRZAQI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius= 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ABUCNR5SXZYM05BZ3NGTUWUST0T4TDFSZPZP5FBX55IZJDRR
CLIENT_SECRET:ZAL0MIKYUWEA0NZDQTD4MXLPAAHZ5DRMNF4L54YW2BHRZAQI


### Get Rome venues with a radius of 1.5km

In [57]:
radius=1500
LIMIT = 400

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [58]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2104, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Rome R. XIV Borgo‎,41.90322,12.49565,Piazza della Repubblica,41.902422,12.496367,Plaza
1,Rome R. XIV Borgo‎,41.90322,12.49565,Museo delle Terme di Diocleziano,41.902912,12.498882,History Museum
2,Rome R. XIV Borgo‎,41.90322,12.49565,Culinaria,41.903718,12.499610,Italian Restaurant
3,Rome R. XIV Borgo‎,41.90322,12.49565,The St. Regis Rome,41.904072,12.494873,Hotel
4,Rome R. XIV Borgo‎,41.90322,12.49565,Come Il Latte,41.907164,12.495989,Ice Cream Shop


In [59]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Rome R. I Monti‎,100,100,100,100,100,100
Rome R. II Trevi‎,100,100,100,100,100,100
Rome R. III Colonna‎,4,4,4,4,4,4
Rome R. IV Campo Marzio‎,100,100,100,100,100,100
Rome R. IX Pigna‎,100,100,100,100,100,100
Rome R. V Ponte‎,100,100,100,100,100,100
Rome R. VI Parione‎,100,100,100,100,100,100
Rome R. VII Regola‎,100,100,100,100,100,100
Rome R. VIII Sant'Eustachio‎,100,100,100,100,100,100


### How many categories exist?

In [61]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 102 uniques categories.


In [62]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Plaza', 'History Museum', 'Italian Restaurant', 'Hotel',
       'Ice Cream Shop', 'Pizza Place', 'Art Museum', 'Church', 'Winery',
       'Trattoria/Osteria', 'Roman Restaurant', 'Coffee Shop', 'Wine Bar',
       'American Restaurant', 'Fountain', 'Pastry Shop', 'Market',
       'Seafood Restaurant', 'Monument / Landmark', 'Café', 'Museum',
       'Greek Restaurant', 'Peruvian Restaurant', 'Pub', 'Jazz Club',
       'Bistro', 'Wine Shop', 'Shopping Mall', 'Bakery', 'Sandwich Place',
       'Bar', 'Boutique', 'Historic Site', 'Chinese Restaurant',
       'Juice Bar', 'Garden', 'Dessert Shop',
       'Vegetarian / Vegan Restaurant', 'Restaurant', 'Scenic Lookout',
       'Temple', 'Road', 'Gourmet Shop', 'Bookstore', 'Beer Bar',
       'Lounge', 'Sushi Restaurant', 'Other Nightlife', 'Deli / Bodega',
       'Japanese Restaurant'], dtype=object)

In [64]:
# check if the results contain "Shopping Mall"
"Restaurant" in venues_df['VenueCategory'].unique()

True

### Okay we got Restaurant in Rome

In [65]:
# one hot encoding
rome_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rome_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [rome_onehot.columns[-1]] + list(rome_onehot.columns[:-1])
rome_onehot = rome_onehot[fixed_columns]

print(rome_onehot.shape)
rome_onehot.head()

(2104, 103)


,Neighborhoods,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,...,Tour Provider,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery
0,Rome R. XIV Borgo‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rome R. XIV Borgo‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rome R. XIV Borgo‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rome R. XIV Borgo‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rome R. XIV Borgo‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [68]:
rome_grouped = rome_onehot.groupby('Neighborhoods').mean().reset_index()
rome_grouped

,Neighborhoods,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,...,Tour Provider,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery
0,Rome R. I Monti‎,0.00,0.00,0.03,0.00,0.00,0.04,0.01,0.00,0.00,...,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00
1,Rome R. II Trevi‎,0.00,0.01,0.03,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.02,0.00,0.00
2,Rome R. III Colonna‎,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00
3,Rome R. IV Campo Marzio‎,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,...,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.01,0.01,0.00
4,Rome R. IX Pigna‎,0.00,0.01,0.03,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.02,0.01,0.01
5,Rome R. V Ponte‎,0.00,0.01,0.03,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.02,0.01,0.01
6,Rome R. VI Parione‎,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.01,...,0.00,0.01,0.00,0.00,0.02,0.00,0.01,0.03,0.00,0.00
7,Rome R. VII Regola‎,0.00,0.00,0.00,0.00,0.00,0.02,0.03,0.00,0.02,...,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.03,0.00,0.00
8,Rome R. VIII Sant'Eustachio‎,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,...,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00
9,Rome R. X Campitelli‎,0.00,0.00,0.01,0.00,0.00,0.02,0.01,0.00,0.01,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.06,0.00,0.00


In [70]:
len(rome_grouped[rome_grouped["Restaurant"] > 0])

18

### Print top 5 venues for each neighbourhood

In [76]:
num_top_venues = 5

for hood in rome_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = rome_grouped[rome_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Rome R. I Monti‎ ----
                 venue  freq
0        Historic Site  0.12
1                Plaza  0.11
2  Monument / Landmark  0.09
3   Italian Restaurant  0.06
4       Ice Cream Shop  0.06


----Rome R. II Trevi‎ ----
            venue  freq
0           Plaza  0.15
1           Hotel  0.10
2  Ice Cream Shop  0.09
3   Historic Site  0.08
4        Fountain  0.07


----Rome R. III Colonna‎ ----
                venue  freq
0  Italian Restaurant  0.25
1           BBQ Joint  0.25
2       Train Station  0.25
3         Supermarket  0.25
4         Music Venue  0.00


----Rome R. IV Campo Marzio‎ ----
                 venue  freq
0   Italian Restaurant  0.12
1                Plaza  0.11
2                Hotel  0.11
3       Ice Cream Shop  0.09
4  Monument / Landmark  0.04


----Rome R. IX Pigna‎ ----
                venue  freq
0               Hotel  0.16
1  Italian Restaurant  0.14
2      Ice Cream Shop  0.08
3               Plaza  0.08
4          Art Museum  0.03


----Rome R. V Pont

In [77]:
rome_restaurant = rome_grouped[["Neighborhoods","Italian Restaurant"]]

In [78]:
rome_restaurant.head()

,Neighborhoods,Italian Restaurant
0,Rome R. I Monti‎,0.06
1,Rome R. II Trevi‎,0.05
2,Rome R. III Colonna‎,0.25
3,Rome R. IV Campo Marzio‎,0.12
4,Rome R. IX Pigna‎,0.14


# CLuster TIME!

In [81]:
# set number of clusters
kclusters = 5

rome_grouped_clustering = rome_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rome_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 0, 0, 0, 2, 3, 2, 2], dtype=int32)

In [82]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
rome_merged = rome_restaurant.copy()

# add clustering labels
rome_merged["Cluster Labels"] = kmeans.labels_

In [87]:
# merge rome grouped with rome_ to add latitude/longitude for each neighborhood
rome_merged = rome_merged.join(df.set_index('Neighborhood'), on='Neighborhoods')

rome_merged.head() 

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Rome R. I Monti‎,0.06,2,41.893150,12.488250
1,Rome R. II Trevi‎,0.05,2,41.901190,12.484460
2,Rome R. III Colonna‎,0.25,1,41.836429,12.756938
3,Rome R. IV Campo Marzio‎,0.12,0,41.907100,12.477860
4,Rome R. IX Pigna‎,0.14,0,41.903220,12.495650


In [88]:
# sort the results by Cluster Labels
print(rome_merged.shape)
rome_merged.sort_values(["Cluster Labels"], inplace=True)
rome_merged

(22, 5)


,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
21,Rome R. XXII Prati‎,0.14,0,41.903220,12.495650
19,Rome R. XX Testac,0.14,0,41.903220,12.495650
3,Rome R. IV Campo Marzio‎,0.12,0,41.907100,12.477860
4,Rome R. IX Pigna‎,0.14,0,41.903220,12.495650
5,Rome R. V Ponte‎,0.14,0,41.903220,12.495650
18,Rome R. XVIII Castro Pretorio‎,0.16,0,41.905510,12.501880
17,Rome R. XVII Sallustiano‎,0.13,0,41.907830,12.496130
16,Rome R. XVI Ludovisi‎,0.11,0,41.907760,12.489550
14,Rome R. XIX Celio‎,0.14,0,41.903220,12.495650
13,Rome R. XIV Borgo‎,0.14,0,41.903220,12.495650


In [91]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import numpy as np
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(rome_merged['Latitude'], rome_merged['Longitude'], rome_merged['Neighborhoods'], rome_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [92]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Result Discussion

## Cluster 1

In [95]:
rome_merged.loc[rome_merged['Cluster Labels'] == 0]

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
21,Rome R. XXII Prati‎,0.14,0,41.90322,12.49565
19,Rome R. XX Testac,0.14,0,41.90322,12.49565
3,Rome R. IV Campo Marzio‎,0.12,0,41.90710,12.47786
4,Rome R. IX Pigna‎,0.14,0,41.90322,12.49565
5,Rome R. V Ponte‎,0.14,0,41.90322,12.49565
18,Rome R. XVIII Castro Pretorio‎,0.16,0,41.90551,12.50188
17,Rome R. XVII Sallustiano‎,0.13,0,41.90783,12.49613
16,Rome R. XVI Ludovisi‎,0.11,0,41.90776,12.48955
14,Rome R. XIX Celio‎,0.14,0,41.90322,12.49565
13,Rome R. XIV Borgo‎,0.14,0,41.90322,12.49565


## Cluster 2

In [96]:
rome_merged.loc[rome_merged['Cluster Labels'] == 1]

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
2,Rome R. III Colonna‎,0.25,1,41.836429,12.756938


## Cluster 3

In [97]:
rome_merged.loc[rome_merged['Cluster Labels'] == 2]

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Rome R. I Monti‎,0.06,2,41.89315,12.48825
9,Rome R. X Campitelli‎,0.12,2,41.89325,12.48143
8,Rome R. VIII Sant'Eustachio‎,0.08,2,41.89965,12.47491
6,Rome R. VI Parione‎,0.10,2,41.89778,12.47075
1,Rome R. II Trevi‎,0.05,2,41.90119,12.48446
10,Rome R. XI Sant'Angelo‎,0.12,2,41.89355,12.47871


## Cluster 4

In [98]:
rome_merged.loc[rome_merged['Cluster Labels'] == 3]

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
7,Rome R. VII Regola‎,0.19,3,41.89480,12.47028
12,Rome R. XIII Trastevere‎,0.23,3,41.88839,12.46621


## Cluster 5

In [99]:
rome_merged.loc[rome_merged['Cluster Labels'] == 4]

,Neighborhoods,Italian Restaurant,Cluster Labels,Latitude,Longitude
20,Rome R. XXI San Saba‎,0.14,4,41.87985,12.4902
15,Rome R. XV Esquilino‎,0.11,4,41.89403,12.5060


# Important Observation

Most of the restaurant are concentrated in the central area of Rome, with the highest number in cluster 2,3,4 and moderate number in cluster 5. On the other hand, cluster 3 has very low number of restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new restaurant as there is very little to no competition from existing ones. Meanwhile, restaurant in cluster 3 are likely suffering from intense competition due to oversupply and high concentration of sthem. From another perspective, this also shows that the oversupply of restaurant mostly happened in the central area of the city, where the most tourist are concentred. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 3 with little to no competition.. Lastly, property developers are advised to avoid neighborhoods in cluster 3 which already have high concentration of restaurant and suffering from intense competition